In [4]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import db_password2
import psycopg2
import pandas as pd
import tensorflow as tf
db_password2

'ucb2020FINAL'

In [12]:
engine = psycopg2.connect(
    database = "postgres",
    user = "postgres",
    password = db_password1,
    #host = "js-ucbfinalprojectdb.c8s02fywjegc.us-east-2.rds.amazonaws.com",
    host = "dataviz-db.csxrf9ti2aba.us-east-2.rds.amazonaws.com",
    port = 5432)
cursor = engine.cursor()

In [14]:
#use Pandas to see all data in table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=engine)

,table_name,column_name,data_type,table_schema
0,bigquery_hits_products,productcouponcode,character varying,public
1,bigquery_hits_products,browser,character varying,public
2,bigquery_hits_products,productquantity,integer,public
3,bigquery_hits_products,hour,integer,public
4,bigquery_hits_products,step,integer,public
...,...,...,...,...
104,vine_table,total_votes,integer,public
105,vine_table,vine,text,public
106,vine_table,star_rating,integer,public
107,vine_table,helpful_votes,integer,public


In [15]:
#Pull all info into pandas dataframe from RDS dB
sql = """
SELECT *
from bigquery_totals2
"""
bigquery_totals_df = pd.read_sql(sql,con = engine)
#plug NaN's as 0's
bigquery_totals_df = bigquery_totals_df.fillna(0)
bigquery_totals_df.head(n = 10)


,fullvisitorid,visitid,visitnumber,visitstarttime,date,socialengagementtype,channelgrouping,browser,operatingsystem,devicecategory,...,istruedirect,bounces,hits,newvisits,pageviews,sessionqualitydim,totaltransactionrevenue,transactionrevenue,transactions,visits
0,0000062267706107999,1499645960,1,1499645960,2017-07-09,Not Socially Engaged,Affiliates,Chrome,Windows,desktop,...,0,1.0,1,1.0,1.0,1.0,0.0,0.0,0.0,1
1,0000085059828173212,1500505105,1,1500505105,2017-07-19,Not Socially Engaged,Direct,Safari,iOS,mobile,...,True,1.0,1,1.0,1.0,1.0,0.0,0.0,0.0,1
2,0000197671390269035,1493772870,1,1493772870,2017-05-02,Not Socially Engaged,Social,Android Webview,Android,mobile,...,0,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0,1
3,000026722803385797,1496658394,1,1496658394,2017-06-05,Not Socially Engaged,Organic Search,Safari,Macintosh,desktop,...,0,0.0,3,1.0,2.0,0.0,0.0,0.0,0.0,1
4,0000436683523507380,1500504900,1,1500504900,2017-07-19,Not Socially Engaged,Referral,Chrome,Macintosh,desktop,...,0,0.0,2,1.0,2.0,1.0,0.0,0.0,0.0,1
5,0000436683523507380,1500989127,2,1500989127,2017-07-25,Not Socially Engaged,Referral,Chrome,Macintosh,desktop,...,True,0.0,8,0.0,8.0,6.0,0.0,0.0,0.0,1
6,0000450371054833295,1500303787,1,1500303787,2017-07-17,Not Socially Engaged,Referral,Chrome,Linux,desktop,...,0,1.0,1,1.0,1.0,1.0,0.0,0.0,0.0,1
7,0000458812883559498,1495066298,1,1495066298,2017-05-17,Not Socially Engaged,Organic Search,Chrome,Linux,desktop,...,0,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0,1
8,0000572434142265465,1494990076,1,1494990076,2017-05-16,Not Socially Engaged,Referral,Chrome,Chrome OS,desktop,...,0,0.0,2,1.0,2.0,0.0,0.0,0.0,0.0,1
9,0000572434142265465,1500605115,2,1500605115,2017-07-20,Not Socially Engaged,Referral,Chrome,Chrome OS,desktop,...,True,0.0,3,0.0,3.0,1.0,0.0,0.0,0.0,1


In [58]:
#Action_type is the column we're going to want to predict.
#This column dictates what kind of action the visitor took on an item

#Drop in Machine Learning logistic regression libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,recall_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics



In [29]:
bigquery_totals_df.dtypes

fullvisitorid               object
visitid                      int64
visitnumber                  int64
visitstarttime               int64
date                        object
socialengagementtype        object
channelgrouping             object
browser                     object
operatingsystem             object
devicecategory              object
continent                   object
subcontinent                object
country                     object
region                      object
metro                       object
city                        object
referralpath                object
source                      object
medium                      object
istruedirect                object
bounces                    float64
hits                         int64
newvisits                  float64
pageviews                  float64
sessionqualitydim          float64
totaltransactionrevenue    float64
transactionrevenue         float64
transactions                 int32
visits              

In [30]:
bigquery_totals_df.nunique()

fullvisitorid              12446
visitid                    15952
visitnumber                  152
visitstarttime             15973
date                          93
socialengagementtype           1
channelgrouping                7
browser                        9
operatingsystem               12
devicecategory                 3
continent                      6
subcontinent                  10
country                       11
region                       145
metro                         46
city                         209
referralpath                 242
source                        10
medium                         6
istruedirect                   2
bounces                        2
hits                          97
newvisits                      2
pageviews                     73
sessionqualitydim             96
totaltransactionrevenue      221
transactionrevenue           202
transactions                   2
visits                         1
dtype: int64

In [31]:
# # Lets work on reducing value dimensions in some of our columns to do oneHotEncoding
# print(bigquery_totals_df.browser.value_counts())
# # We can reduce by counting all browser values under 100 as OTHER

# print(bigquery_totals_df.subcontinent.value_counts())
# # We can reduce by count all subcontinent values under 500 as OTHER

# print(bigquery_totals_df.country.value_counts())
# bigquery_totals_df.country.value_counts().plot.density()
# bigquery_totals_df.country.value_counts()[bigquery_totals_df.country.value_counts()>250]
# # We can reduce by count of all countries values under 250 as OTHER

# print(bigquery_totals_df.region.value_counts())
# bigquery_totals_df.region.value_counts()[bigquery_totals_df.region.value_counts()>100]
# # A lot of "not available in demo dataset". I think we should remove this column.

# bigquery_totals_df.metro.value_counts()
# # A lot of "not available in demo dataset". I think we should remove this column.

# bigquery_totals_df.city.value_counts()
# # A lot of "not available in demo dataset". I think we should remove this column.

# bigquery_totals_df.referralpath.value_counts()
# bigquery_totals_df.referralpath.value_counts()[bigquery_totals_df.referralpath.value_counts()>10]
# # Unsure how well this column will be in our model. I think we should remove this column 

# bigquery_totals_df.source.value_counts()
# bigquery_totals_df.source.value_counts()[bigquery_totals_df.source.value_counts()>50]
# # We can reduce by count of all sources under 50 as OTHER





In [32]:
#REPLACEEEEEEEEEEEEEEE
replace_browser_types = list(bigquery_totals_df.browser.value_counts()[bigquery_totals_df.browser.value_counts()<100].index)
replace_subcontinent_types = list(bigquery_totals_df.subcontinent.value_counts()[bigquery_totals_df.subcontinent.value_counts()<500].index)
replace_country_types = list(bigquery_totals_df.country.value_counts()[bigquery_totals_df.country.value_counts()<250].index)
replace_source_types = list(bigquery_totals_df.source.value_counts()[bigquery_totals_df.source.value_counts()<50].index)


replace_transactions_values = list(bigquery_totals_df.transactions.value_counts().index)
replace_transactions_values

[0, 1]

In [33]:
#Replace in dataframe
for browser_replace in replace_browser_types:
    bigquery_totals_df.browser = bigquery_totals_df.browser.replace(browser_replace,"Other")

for subcontinent_replace in replace_subcontinent_types:
    bigquery_totals_df.subcontinent = bigquery_totals_df.subcontinent.replace(subcontinent_replace,"Other")
    
for country_replace in replace_country_types:
    bigquery_totals_df.country = bigquery_totals_df.country.replace(country_replace,"Other")
    
for source_replace in replace_source_types:
    bigquery_totals_df.source = bigquery_totals_df.source.replace(source_replace,"Other")
    
# Our predicting variable transactions, is an integer value for total number of ecommerce transactions. We want to make this a categorical variable
for trans_vals in replace_transactions_values:
    if trans_vals > 0.0:
        bigquery_totals_df.transactions[bigquery_totals_df.transactions == trans_vals] = 1
    elif trans_vals == 0.0:
        bigquery_totals_df.transactions[bigquery_totals_df.transactions == trans_vals] = 0
    
bigquery_totals_df.transactions = bigquery_totals_df.transactions.astype(int)
#bigquery_totals_df.transactions.unique() 
bigquery_totals_df.nunique() 

C:\Users\Nathan\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Nathan\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


fullvisitorid              12446
visitid                    15952
visitnumber                  152
visitstarttime             15973
date                          93
socialengagementtype           1
channelgrouping                7
browser                        9
operatingsystem               12
devicecategory                 3
continent                      6
subcontinent                  10
country                       11
region                       145
metro                         46
city                         209
referralpath                 242
source                        10
medium                         6
istruedirect                   2
bounces                        2
hits                          97
newvisits                      2
pageviews                     73
sessionqualitydim             96
totaltransactionrevenue      221
transactionrevenue           202
transactions                   2
visits                         1
dtype: int64

In [34]:
#Create a list of the categorical variables
bigquery_totals_cat = bigquery_totals_df.dtypes[bigquery_totals_df.dtypes == "object"].index.tolist()


# From the list of categorical variables we DONT want some of them. We'll .remove() the columns we want from the list
removal_list = ['fullvisitorid','region','metro','city','referralpath','istruedirect','date']

for elem in removal_list:
    bigquery_totals_cat.remove(elem)
# This is pretty ugly. 

# Fix Boolean istruedirect column from True/None to 1/0
# bigquery_totals_df["istruedirect"] = bigquery_totals_df["istruedirect"].astype(int)

# #get number of unique values per category column
bigquery_totals_df[bigquery_totals_cat].head()

,socialengagementtype,channelgrouping,browser,operatingsystem,devicecategory,continent,subcontinent,country,source,medium
0,Not Socially Engaged,Affiliates,Chrome,Windows,desktop,Americas,South America,Other,Partners,affiliate
1,Not Socially Engaged,Direct,Safari,iOS,mobile,Americas,Northern America,United States,(direct),(none)
2,Not Socially Engaged,Social,Android Webview,Android,mobile,Americas,Northern America,United States,m.facebook.com,referral
3,Not Socially Engaged,Organic Search,Safari,Macintosh,desktop,Europe,Northern Europe,United Kingdom,google,organic
4,Not Socially Engaged,Referral,Chrome,Macintosh,desktop,Americas,Northern America,United States,(direct),(none)


In [35]:
#slap a OneHotEncoding on this for categorical columns
enc = OneHotEncoder(sparse=False)

#fit and transform the encoding using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(bigquery_totals_df[bigquery_totals_cat]))

#add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(bigquery_totals_cat)
encode_df.head()

,socialengagementtype_Not Socially Engaged,channelgrouping_Affiliates,channelgrouping_Direct,channelgrouping_Display,channelgrouping_Organic Search,channelgrouping_Paid Search,channelgrouping_Referral,channelgrouping_Social,browser_Android Webview,browser_Chrome,...,source_google,source_google.com,source_m.facebook.com,source_youtube.com,medium_(none),medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [36]:
#merge one-hot encoded features and drop the originals
bigquery_totals_df1 = bigquery_totals_df.merge(encode_df,left_index = True, right_index = True)
bigquery_totals_df1 = bigquery_totals_df1.drop(columns = bigquery_totals_cat+removal_list)
#bigquery_hits_df1 = bigquery_hits_df1.drop(columns = remove_these_columns)
bigquery_totals_df1.head()
#bigquery_hits_df1.productprice

,visitid,visitnumber,visitstarttime,bounces,hits,newvisits,pageviews,sessionqualitydim,totaltransactionrevenue,transactionrevenue,...,source_google,source_google.com,source_m.facebook.com,source_youtube.com,medium_(none),medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral
0,1499645960,1,1499645960,1.0,1,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1500505105,1,1500505105,1.0,1,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1493772870,1,1493772870,1.0,1,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1496658394,1,1496658394,0.0,3,1.0,2.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1500504900,1,1500504900,0.0,2,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [37]:
#Remove Action type from our features dataset
y = bigquery_totals_df1.transactions

X = bigquery_totals_df1.drop(columns =['transactions','totaltransactionrevenue','transactionrevenue'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1337,stratify = y)

#y.unique()

In [73]:
# Upsampling to boost our test variable!
from sklearn.utils import resample

# What we'll do is rejoin our traning data set back together, and then resample data from the training data
# such that we have an equal number of transactions (value of 1) and no transactions (value of 0)

# Concatenate our training data back together
X = pd.concat([X_train,y_train], axis = 1)

# Separate minority and majority classes
not_transaction = X[X.transactions == 0]
transaction = X[X.transactions == 1]

# Upsample minority
transaction_upsampled = resample(transaction,
                                replace = True, # sample with replacement
                                n_samples = len(not_transaction), # match number in majority class
                                random_state = 27) # reproducible results

#combine majority and upsampled minority
upsampled = pd.concat([not_transaction,transaction_upsampled])

# check new transaction counts
upsampled.transactions.value_counts()



1    11828
0    11828
Name: transactions, dtype: int64

In [147]:
# Undersampling to even our test variable!
# use the same tranasction and not_transaction from above

# downsampled majority
not_transaction_downsampled = resample(not_transaction,
                                replace = False, # sample without replacement
                                n_samples = len(transaction), # match number in minority class
                                random_state = 27) # reproducible results

# Combine minority and downsampled majority
downsampled = pd.concat([not_transaction_downsampled,transaction])

# Checking counts
downsampled.transactions.value_counts()


1    172
0    172
Name: transactions, dtype: int64

In [149]:
# Point to our upsampled/downsampled data
# Upsampled
# y_train = upsampled.transactions
# X_train = upsampled.drop('transactions',axis = 1)

# Downsampled
y_train = downsampled.transactions
X_train = downsampled.drop('transactions',axis = 1)

In [150]:
#Standar scaler time lets get it

#create a StandardScaler instance
scaler = StandardScaler()

#Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [151]:
# Function for all Machine Learning methods. This will help compare results
def run_ml_model(x_data_train,x_data_test,y_data_train,y_data_test,model,n_runs):
    if model == 'Logistic Regression':
        # Define the logistic regression model
        log_classifier = LogisticRegression(solver="lbfgs",max_iter=n_runs)

        # Train the model
        log_classifier.fit(x_data_train,y_data_train)

        # Evaluate the model
        y_pred = log_classifier.predict(x_data_test)

        my_cm = metrics.confusion_matrix(y_data_test,y_pred)
        my_acc = accuracy_score(y_data_test,y_pred)
        my_f1  = f1_score(y_data_test,y_pred)
        my_recall = recall_score(y_data_test,y_pred)
        
    if model == 'Random Forest':
        # Create a random forest classifier.
        rf_model = RandomForestClassifier(n_estimators=n_runs, random_state=1337)

        # Fitting the model
        rf_model = rf_model.fit(x_data_train, y_data_train)

        # Evaluate the model
        y_pred = rf_model.predict(x_data_test)
        my_cm = metrics.confusion_matrix(y_data_test,y_pred)
        my_acc = accuracy_score(y_data_test,y_pred)
        my_f1  = f1_score(y_data_test,y_pred)
        my_recall = recall_score(y_data_test,y_pred)
    
    if model == 'Neural Network':
        # Define the basic neural network model
        nn_model = tf.keras.models.Sequential()
        nn_model.add(tf.keras.layers.Dense(units=25, activation="tanh", input_dim=84))
        nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))

        # Compile the Sequential model together and customize metrics
        nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

        # Train the model
        fit_model = nn_model.fit(x_data_train, y_data_train, epochs=n_runs)

        y_pred = nn_model.predict_classes(x_data_test,verbose = 1)

        # Evaluate the model using the test data
        nn_model.evaluate(x_data_test,y_data_test,verbose=0)
        my_cm = metrics.confusion_matrix(y_data_test,y_pred)
        my_acc = accuracy_score(y_data_test,y_pred)
        my_f1  = f1_score(y_data_test,y_pred)
        my_recall = recall_score(y_data_test,y_pred)
        
    #return print(cm), print(f" Logistic regression model accuracy: {accuracy_score(y_data_test,y_pred):.3f}"), print(f" F1 Score: {f1_score(y_data_test,y_pred):.3f}"),print(f" Recall Score: {recall_score(y_data_test,y_pred):.3f}")
    return my_cm, my_acc, my_f1,my_recall

In [153]:
print('====== Logistic Regression')
log_cm,log_acc,log_f1,log_recall = run_ml_model(X_train_scaled,X_test_scaled,y_train,y_test,'Logistic Regression',n_runs = 10000)
print(log_cm,'\n',f" Logistic regression model accuracy: {log_acc:.3f}",'\n',f" F1 Score: {log_f1:.3f}",'\n',f" Recall Score: {log_recall:.3f}")
print('===========================\n')

print('====== Random Forest')
rf_cm,rf_acc,rf_f1,rf_recall = run_ml_model(X_train_scaled,X_test_scaled,y_train,y_test,'Random Forest',n_runs = 350)
print(rf_cm,'\n',f" Random Forest model accuracy: {rf_acc:.3f}",'\n',f" F1 Score: {rf_f1:.3f}",'\n',f" Recall Score: {rf_recall:.3f}")
print('===========================\n')

print('====== Neural Network')
nn_cm,nn_acc,nn_f1,nn_recall = run_ml_model(X_train_scaled,X_test_scaled,y_train,y_test,'Random Forest',n_runs = 200)
print(nn_cm,'\n',f" Neural Network model accuracy: {nn_acc:.3f}",'\n',f" F1 Score: {nn_f1:.3f}",'\n',f" Recall Score: {nn_recall:.3f}")
print('===========================\n')



====== Logistic Regression
[[3554  388]
 [   1   57]] 
  Logistic regression model accuracy: 0.903 
  F1 Score: 0.227 
  Recall Score: 0.983

====== Random Forest
[[3673  269]
 [   1   57]] 
  Random Forest model accuracy: 0.932 
  F1 Score: 0.297 
  Recall Score: 0.983

====== Neural Network
[[3672  270]
 [   1   57]] 
  Neural Network model accuracy: 0.932 
  F1 Score: 0.296 
  Recall Score: 0.983



# Scratchwork below

In [82]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=10000)

# Train the model
log_classifier.fit(X_train_scaled,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test_scaled)

cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)

print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")



print(f" F1 Score: {f1_score(y_test,y_pred):.3f}")
print(f" Recall Score: {recall_score(y_test,y_pred):.3f}")

[[3727  215]
 [   6   52]]
 Logistic regression model accuracy: 0.945
 F1 Score: 0.320
 Recall Score: 0.897


In [83]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=350, random_state=1337)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)


print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

print(f" F1 Score: {f1_score(y_test,y_pred):.3f}")
print(f" Recall Score: {recall_score(y_test,y_pred):.3f}")

[[3915   27]
 [  42   16]]
 Random forest predictive accuracy: 0.983
 F1 Score: 0.317
 Recall Score: 0.276


In [84]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=5, activation="tanh", input_dim=84))
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

y_pred = nn_model.predict_classes(X_test_scaled,verbose = 1)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")





Train on 23656 samples
Epoch 1/50
23656/23656 [==============================] - 1s 47us/sample - loss: 1.0744 - accuracy: 0.8340
Epoch 2/50
23656/23656 [==============================] - 1s 23us/sample - loss: 0.4172 - accuracy: 0.9382
Epoch 3/50
23656/23656 [==============================] - 1s 22us/sample - loss: 0.3283 - accuracy: 0.9559
Epoch 4/50
23656/23656 [==============================] - 1s 22us/sample - loss: 0.3289 - accuracy: 0.9584
Epoch 5/50
23656/23656 [==============================] - 1s 22us/sample - loss: 0.3324 - accuracy: 0.9579
Epoch 6/50
23656/23656 [==============================] - 1s 22us/sample - loss: 0.3096 - accuracy: 0.9629
Epoch 7/50
23656/23656 [==============================] - 1s 23us/sample - loss: 0.3120 - accuracy: 0.9629
Epoch 8/50
23656/23656 [==============================] - 1s 22us/sample - loss: 0.3049 - accuracy: 0.9650
Epoch 9/50
23656/23656 [==============================] - 1s 22us/sample - loss: 0.3168 - accuracy: 0.9628
Epoch 10/50
23

In [86]:
cm = metrics.confusion_matrix(y_test,y_pred)
print(cm)
print(f" Neural Network predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")
print(f" F1 Score: {f1_score(y_test,y_pred):.3f}")
print(f" Recall Score: {recall_score(y_test,y_pred):.3f}")

[[3785  157]
 [  10   48]]
 Neural Network predictive accuracy: 0.958
 F1 Score: 0.365
 Recall Score: 0.828
